In [1]:
import os
from PIL import Image

In [2]:
os.mkdir("./data")

In [3]:
os.mkdir("./data/fake")
os.mkdir("./data/real")

In [4]:
folder_path="/kaggle/input/stable-diffusion-generated-images/StableDiff/StableDiff/StableDiff"

In [5]:
file_names=os.listdir(folder_path)

In [6]:
fake_path="./data/fake"

In [7]:
for file in file_names:
    img=Image.open(folder_path+"/"+file)
    img.save(fake_path+"/"+file+".JPEG")

In [8]:
len(os.listdir("./data/fake"))

4952

In [9]:
folder_path="/kaggle/input/stable-diffusion-generated-images/laion400m-laion4.75/laion400m-laion4.75/laion400m-laion4.75+/laion400m-laion4.75+"

In [10]:
file_names=os.listdir(folder_path)

In [11]:
for file in file_names:
    try:
        img=Image.open(folder_path+"/"+file)
        if img.format == 'PNG':
            if img.mode != 'RGBA':
                img=img.convert("RGBA")
        img=img.convert("RGB")
        img.save(fake_path+"/"+file+".JPEG")
    except:
        pass

In [12]:
len(os.listdir(fake_path))

9997

In [13]:
folder_path="/kaggle/input/imagenet/imagenet/train"

In [14]:
file_names=os.listdir(folder_path)
fake_path="./data/real"

In [15]:
i=0
for file in file_names:
    img=Image.open(folder_path+"/"+file)
    img.save(fake_path+"/"+file+".JPEG")
    i=i+1
    if i==9997:
        break

In [16]:
len(os.listdir("./data/real"))

9997

In [17]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [18]:
from tensorflow import keras

In [19]:
from keras.applications import VGG16
from keras.models import Sequential
from keras import layers
from keras.layers import Flatten,Dropout

In [20]:
vgg_base=VGG16(weights='imagenet',include_top=False,input_shape=(256,256,3))

58889256/58889256 [==============================] - 3s 0us/step


In [21]:
model=Sequential(
    [
        layers.Rescaling(1/255.0),
        vgg_base,
        Flatten(),
        Dropout(0.5),
        layers.Dense(512,activation='relu'),
        Dropout(0.2),
        layers.Dense(256,activation='relu'),
        Dropout(0.10),
        layers.Dense(2),
    ]
)

In [22]:
vgg_base.trainable=False

In [23]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [24]:
img_height,img_width=256,256
batch_size=256

In [25]:
train_ds=keras.utils.image_dataset_from_directory("./data",image_size=(img_height,img_width),batch_size=batch_size)

Found 19994 files belonging to 2 classes.


In [26]:
model.fit(train_ds,batch_size=batch_size,epochs=50,verbose=2)

Epoch 1/50
79/79 - 100s - loss: 0.6134 - accuracy: 0.6659 - 100s/epoch - 1s/step
Epoch 2/50
79/79 - 70s - loss: 0.4958 - accuracy: 0.7557 - 70s/epoch - 892ms/step
Epoch 3/50
79/79 - 70s - loss: 0.4568 - accuracy: 0.7812 - 70s/epoch - 885ms/step
Epoch 4/50
79/79 - 70s - loss: 0.4352 - accuracy: 0.7923 - 70s/epoch - 884ms/step
Epoch 5/50
79/79 - 70s - loss: 0.4075 - accuracy: 0.8088 - 70s/epoch - 886ms/step
Epoch 6/50
79/79 - 70s - loss: 0.3917 - accuracy: 0.8191 - 70s/epoch - 887ms/step
Epoch 7/50
79/79 - 70s - loss: 0.3704 - accuracy: 0.8318 - 70s/epoch - 887ms/step
Epoch 8/50
79/79 - 70s - loss: 0.3549 - accuracy: 0.8402 - 70s/epoch - 891ms/step
Epoch 9/50
79/79 - 69s - loss: 0.3337 - accuracy: 0.8516 - 69s/epoch - 875ms/step
Epoch 10/50
79/79 - 70s - loss: 0.3241 - accuracy: 0.8580 - 70s/epoch - 882ms/step
Epoch 11/50
79/79 - 70s - loss: 0.3139 - accuracy: 0.8610 - 70s/epoch - 885ms/step
Epoch 12/50
79/79 - 69s - loss: 0.2891 - accuracy: 0.8715 - 69s/epoch - 875ms/step
Epoch 13/50
79

In [27]:
model.save("./save")